In [17]:
import pandas as pd
import numpy as np

import string
import re
import nltk
from nltk.corpus import stopwords, brown
from nltk.tokenize import word_tokenize 
from rake_nltk import Rake, Metric
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import bs4
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.request import Request, urlopen 
import requests
import pycountry
import os



In [18]:
info=0
def info_finder(name):
    global info
    prod_name=name
    page_html = 'https://www.google.com/search?q=splendor+bike+alternative'
    info=[]
    res = requests.get(page_html)
    soup = bs4.BeautifulSoup(res.text)
    # table=soup.find_all('tr')

    item=soup.find_all('div',class_='.TrT0Xe , .TrT0Xe b')
    
    return item

In [19]:
page_html = 'https://www.google.com/search?q=splendor+bike+alternative'
info=[]
res = requests.get(page_html)
soup = bs4.BeautifulSoup(res.text)
for i in soup.find_all('div',class_='BNeawe s3v9rd AP7Wnd')[:10]:
    print(i.text)

Find FAQ of Splendor Plus AlternativesHero HF Deluxe. 48,950 onwards* HF Deluxe FAQ. Splendor Plus vs HF Deluxe.Hero Passion Pro. 66,500 onwards* Passion Pro FAQ. ... TVS Radeon. 60,442 onwards* Radeon FAQ. ... 69,415 onwards* Shine FAQ. Splendor Plus vs Shine.Bajaj Pulsar 125 Neon. 72,122 onwards* Pulsar 125 Neon FAQ. ... Bike Comparision.
Find FAQ of Splendor Plus Alternatives
Hero HF Deluxe. 48,950 onwards* HF Deluxe FAQ. Splendor Plus vs HF Deluxe.
Hero Passion Pro. 66,500 onwards* Passion Pro FAQ. ... 
TVS Radeon. 60,442 onwards* Radeon FAQ. ... 
69,415 onwards* Shine FAQ. Splendor Plus vs Shine.
Bajaj Pulsar 125 Neon. 72,122 onwards* Pulsar 125 Neon FAQ. ... 
Bike Comparision.
Bikes with Similar Price. Hero HF Deluxe. Rs. 48,950. Splendor Plus vs HF Deluxe. Honda Shine. Rs. 69,415. Splendor Plus vs Shine. Hero Passion Pro. Rs. 66,500. Bajaj Pulsar 125 Neon. Rs. 72,122. Rs. 71,000. Splendor Plus vs Glamour. Hero Super Splendor. Rs. 69,450. Bajaj Platina 100. Rs. 50,592. Honda Livo

In [20]:
countries_cities=['beijing','taiwan','guangdon','american','america']

for x in pycountry.countries:
    countries_cities.append(x.name.lower())
countries_cities.remove('india')
countries_cities.remove('british indian ocean territory')

for city in pycountry.subdivisions:
    if 'IN-' not in city.code:
        if city.name=='London, City of':
            countries_cities.append('london')
        else:
            countries_cities.append(city.name.lower())
langs=[]            
for lang in pycountry.languages:
    if 'India' not in lang.name:
        langs.append(lang.name.lower())
countries_cities.extend(langs)

In [21]:
details=0
def origin_finder(name):
    global details
    company_name=name
    page_html = 'https://www.google.com/search?q='+company_name+'+company+belongs+to+which+country'
    details=[]
    try:
        res = requests.get(page_html)
    except:
        print('Error in Fetching from Google Server.')

    soup = bs4.BeautifulSoup(res.text)
    # table=soup.find_all('tr')

    item=soup.find_all('div',class_='AP7Wnd')
    for info in item:
        details.append(info.get_text().lower())
#     print(' '.join(details[:11]))
    details=' '.join(details[:11])
    if 'india'in details[:10] :
        return name
    else:
        for foreign_cnt in countries_cities:
            if len(foreign_cnt)>=5:
                if foreign_cnt in details:
                    print(foreign_cnt)
                    break
        else:
            return name
    return None
    
# company= []
# for row in table:
#     data=i.find_all('td')
#     row=[j.text for j in data]
#     company.append(row.text)

# 'india' in ''.join(company)
# print(company)
#########################################################

# rQMQod
# AP7Wnd


# PROD-DETAILS

In [22]:
def prod_details(prod_name):
    products=[] #List to store name of the product
    prices=[] #List to store price of the product
    ratings=[] #List to store rating of the product
    categories=[]
    prod_url=[]
    brands=[]
    
    for pgno in range(1):
        create_url = "https://www.flipkart.com/search?q="+prod_name+'&page='+str(pgno)
        res = requests.get(create_url)
        # content = driver.page_source
        soup = BeautifulSoup(res.text)
        
        
        #NAME
        for n in soup.findAll('a', attrs={'class':'s1Q9rs'}):
            name=n.text
            products.append(name) 
            brands.append(name.split(' ')[0])
        if len(products)==0:
            return pd.DataFrame()
        
        # PRICES
        for p in soup.findAll('div',attrs={'class':'_25b18c'}):
            price=p.find('div', attrs={'class':'_30jeq3'})
            try:
                prices.append(price.text)
            except:
                pass
        
        if len(prices)==0:
            return pd.DataFrame()
        
        #RATINGS
        
        for r in soup.findAll('div',attrs={'class':'_3LWZlK'}):
            rating = r.text
            ratings.append(rating)
            
        if len(ratings)==0:
            return pd.DataFrame()    
            
        #CATEGORY
        
        category = soup.findAll('a',attrs={'class':'_2whKao'})[-1].text
        #URL
        
        for u in soup.findAll('a',href=True, attrs={'class':'s1Q9rs'}):
            url=u.attrs['href']
            prod_url.append('https://www.flipkart.com'+url)
            categories.append(category)
        if len(prod_url)==0:
            return pd.DataFrame()
        
        
           
    return pd.DataFrame.from_dict({'Product Name':products,'Brand':brands, 'Price':prices, 'Ratings':ratings, 'Category':categories, 'URL':prod_url},orient='index').transpose().dropna()

# PROD-DETAILS-2

In [23]:
def prod_details_2(prod_name):   
    
    products=[]
    prices=[]
    ratings=[]
    categories=[]
    prod_url=[]
    brands=[]
    create_url = "https://www.flipkart.com/search?q="+prod_name
    res = requests.get(create_url)
    soup = BeautifulSoup(res.text)
    category = category = soup.findAll('a',attrs={'class':'_2whKao'})[-1].text
    print(category)
    for pgno in range(1):
        create_url = "https://www.flipkart.com/search?q="+prod_name+'&page='+str(pgno)
        res = requests.get(create_url)
        soup = BeautifulSoup(res.text)
        try:
            for a in soup.findAll('div', attrs={'class':'_3pLy-c row'}):
                name=a.findAll('div', attrs={'class':'_4rR01T'})[0].text
                products.append(name)
                if len(products)==0:
                    return pd.DataFrame()
                brands.append(name.split(' ')[0])
                
                price=a.findAll('div', attrs={'class':'_30jeq3 _1_WHN1'})[0].text
                prices.append(price)
                if len(prices)==0:
                    return pd.DataFrame()
                
                rating=a.findAll('div', attrs={'class':'_3LWZlK'})[0].text
                ratings.append(rating)
                if len(ratings)==0:
                    return pd.DataFrame()
                
                
                categories.append(category)
                if len(categories)==0:
                    return pd.DataFrame()
            for a in soup.findAll('a', attrs={'class':'_1fQZEK'}):
                    url=a.attrs['href']
                    prod_url.append('https://www.flipkart.com'+url)
                
                
        except:
            print('Error fetching some details.')

    return pd.DataFrame.from_dict({'Product Name':products,'Brand':brands, 'Price':prices, 'Ratings':ratings, 'Category':categories, 'URL':prod_url},orient='index').transpose().dropna()

# PROD-WEAR

In [24]:
def prod_wear(prod_name):
    products=[] #List to store name of the product
    prices=[] #List to store price of the product
    ratings=[] #List to store rating of the product
    categories=[]
    prod_url=[]
    brands=[]

    for pgno in range(1):
        create_url = "https://www.flipkart.com/search?q="+prod_name+'&page='+str(pgno)
        res = requests.get(create_url)
        soup = BeautifulSoup(res.text)
        
        
        #BRANDS
        for b in soup.findAll('div',attrs={'class':'_2WkVRV'}):
            brands.append(b.text)
        
        
        #NAME
        if len(brands)==0:
            for n in soup.findAll('a',href=True, attrs={'class':'IRpwTa'}):
                name=n.text
                products.append(name)
                brands.append(name.split(' ')[0])
        else:
            for n in soup.findAll('a',href=True, attrs={'class':'IRpwTa'}):
                name=n.text
                products.append(name)
        

        # PRICES
        for p in soup.findAll('div',attrs={'class':'_25b18c'}):
            price=p.find('div', attrs={'class':'_30jeq3'})
            prices.append(price.text)
#         #RATINGS

#         for r in soup.findAll('div',attrs={'class':'hGSR34'}):
#             rating = r.text
#             ratings.append(rating)
        #CATEGORY

        category = soup.findAll('a',attrs={'class':'_2whKao'})[-1].text
        #URL

        for u in soup.findAll('a',href=True, attrs={'class':'_2UzuFa'}):
            url=u.attrs['href']
            prod_url.append('https://www.flipkart.com'+url)
            categories.append(category)
        
           
    return pd.DataFrame.from_dict({'Product Name':products, 'Brand':brands, 'Price':prices, 'Category':categories, 'URL':prod_url},orient='index').transpose().dropna()

# FETCH-DETAILS

In [25]:
def fetch_details(prod_name):
    prod_df = prod_details(prod_name)
    if len(prod_df)==0:
        prod_df = prod_details_2(prod_name)
    if len(prod_df)==0:
        prod_df = prod_wear(prod_name)
    return prod_df

In [26]:
def text_preprocess(col):
    try:
        text = col.lower()
        return text
    except:
        return col

def price_conversion(col):
    try:
        translation_tabel=str.maketrans({'₹':"",',':""})
        return float(col.translate(translation_tabel))
    except:
        return col
    

def cleaning(string):
    text = re.sub('[^A-Za-z]+', ' ', string) # removing numbers and special characters
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))] #removing stop-words
    return ' '.join(text)

In [27]:
def prod_insert(col,products):
    newprod = {'Product Name':col,'Brand':col.split(' ')[0],'Category':col.split(' ')[-1]}
    if newprod['Product Name'] not in products['Product Name'].to_numpy():
        mod_products = products.append(newprod,ignore_index=True)
        return mod_products
    else:
        print('YA')
        return products
    
def key_details(col):    
    product_name = col[0]
    category = col[1]
    features = ' '.join(product_name.split()[1:])+' '+category
    key_words = ' '.join(list(set(features.split())))
    return key_words


def confidence_rating(col,info):    
    
    r = Rake(max_length=5,min_length=0) # Uses stopwords for english from NLTK, and all puntuation characters.
    r.extract_keywords_from_text(info)
    det = ' '.join(r.get_ranked_phrases())
    for kw in col.split():
        if kw in det:
            return 1
    return 0


In [28]:
tf = TfidfVectorizer(analyzer='word',stop_words='english')

In [29]:

def recommendation_engine(prod_name,products):
    global glob_prod,i
    prod_name=prod_name.lower()
    
    try:

        brands=list(products['Brand'].value_counts().index)
        print(len(brands))
        brands_origin = {}
        for brand in brands[:5]:
            brands_origin[brand]=origin_finder(brand)
            
        products['Brand'] = products["Brand"].replace(brands_origin)
        products.dropna(axis=0,inplace=True)

        products=prod_insert(prod_name,products)
        try:
            items = products[['Product Name','Category','Brand','Ratings','Price','URL']]    
        except:
            items = products[['Product Name','Category','Brand','Price','URL']]   
            
        products['Key Words'] = products[['Product Name','Category']].apply(key_details,axis=1)
       
        products = products.applymap(text_preprocess)

        products.drop_duplicates(inplace=True,ignore_index=True,keep='first',subset='Product Name')

        glob_prod = products
        tfidf_matrix = tf.fit_transform(products['Key Words'])
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        indices = pd.Series(products.index, index=products['Product Name']) 

        idx = indices[prod_name]

        sim_scores = list(enumerate(cosine_sim[idx]))

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        sim_scores = sim_scores[1:20]
        prod_indices = [i[0] for i in sim_scores]

        return pd.DataFrame(items.iloc[prod_indices])
    except:
        return {'status':0,'message':"No Results Found"}

In [30]:
def get_product(prod_name, prod_type):   
    user_search_list = open(os.getcwd()+'\\user_history.txt',"a")
    user_search_list.write('\n'+prod_name+'\t'+prod_type)
    user_search_list.close()
    File_object = open(os.getcwd()+'\\eng_words.txt',"r")
    eng_words = File_object.read().replace('\n',' ').split()
    name = prod_name.split()
    lemmatizer = WordNetLemmatizer()
    prod_name = ' '.join([lemmatizer.lemmatize(i) for i in name])
#     print(prod_name)
    try:
        if prod_type=='':
            a=[]
            for i in prod_name.split():
                if i in eng_words:
                    print(i)
                    a.append(i)
            search_q = ' '.join(a)
            if len(search_q)==0:
                search_q = prod_name.split()[-1]
        else:
            search_q = prod_type
        print(search_q)
        products = fetch_details(search_q)
        products.fillna('',inplace=True)

        recom_prod = recommendation_engine(prod_name,products)
        if type(recom_prod)==str:
            return recom_prod
        else:
            print(recom_prod)
            return recom_prod
    except:
        return {'status':0,'message':"No Results Found"}
    

In [31]:
# from flask import Flask,jsonify, request

# app = Flask(__name__)
# @app.route("/product")
# def prod_Rcom():
#     prod_name = request.args['prodname']
#     prod_type = request.args['prodtype']
#     return get_product(prod_name,prod_type)

# @app.route("/")
# def start_req():
#     return 'Service Started'

# if __name__ == '__main__':
#     app.run(debug=False)


In [38]:
import base64
import datetime
import io

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash_table.Format import Format, Group, Scheme, Symbol
import dash_bootstrap_components as dbc

image_bg = 'D://Work Files//Compcarta//assets//bg.jpg'
encoded_bg = base64.b64encode(open(image_bg, 'rb').read())

app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])
app.layout = html.Div([
    html.H1('Product Recommender',className='text-field-title'),
    dbc.Input(id="product-name", placeholder="Enter the Name of Product", type="text",style = dict(width = '20%',display = 'list-item'),className='text-field'),
    dbc.Button("Find Similar Products", id='calculate',color="success",size="lg",n_clicks=0),
    html.Div(id='similar-products'),
    
    html.Div( children=[
    html.Img(className='bg',src='data:image/png;base64,{}'.format(encoded_bg.decode())) ])
    
])



def dash_table_parser(df):
    link=[]
    for url in df['URL']:
        link.append(html.A(html.P('Link'),href=url,target="_blank"))
    dict_link={"Link":link}
    link_df=pd.DataFrame(dict_link)
    df = df.merge(link_df,on=df.index)
    df.drop(['URL','key_0'],axis=1,inplace=True)
    return html.Div([html.H6('The Confidence Rating is :'), dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True)])


@app.callback(Output('similar-products', 'children'),
              [Input('calculate', 'n_clicks')],
              [State('product-name', 'value')])
def success(n_clicks,prod_name):
    if n_clicks > 0:
        
        recom_prod = get_product(prod_name,'')
        
        return [dash_table_parser(recom_prod)]
    

if __name__ == '__main__':
    app.run_server(debug=False)



Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


deodarant
14
north
american


127.0.0.1 - - [08/Dec/2020 00:03:24] "POST /_dash-update-component HTTP/1.1" 200 -


australia
                                         Product Name    Category   Brand  \
0       Vanesa Babe Deo Deodorant Spray  -  For Women  Deodorants  Vanesa   
1   Vanesa Deodorant Touche Deo 150 Ml Deodorant S...  Deodorants  Vanesa   
2   Park Avenue Good Morning , Cool Blue & Storm D...  Deodorants    Park   
3   Park Avenue Signature - Voyage, Neo Deodorant ...  Deodorants    Park   
4   Vanesa Envy 1000 Magnetic Deodorant Spray  -  ...  Deodorants  Vanesa   
5   Vanesa Petal Deo 150 Ml Deodorant Spray  -  Fo...  Deodorants  Vanesa   
6   Park Avenue Good Morning Deodorant Spray  -  F...  Deodorants    Park   
7   AXE Dark Temptation Long Lasting Deodorant Bod...  Deodorants     AXE   
8   Brut DEODORANT IDENTITY 200 ML Deodorant Spray...  Deodorants    Brut   
9   Set Wet Charm Avatar Deodorant & Body Spray Pe...  Deodorants     Set   
10  Park Avenue Body - Good Morning, Storm Deodora...  Deodorants    Park   
11  Vanesa Shero and Queen Combo (Pack of 2) Deodo...  Deodorants 

...

Biscuits
11


127.0.0.1 - - [08/Dec/2020 00:05:01] "POST /_dash-update-component HTTP/1.1" 200 -


north
                                         Product Name  \
26          Britannia Nutri Choice Digestive Biscuits   
15                              Parle G Gold Biscuits   
2                       Britannia Marie Gold Biscuits   
6                           Oreo Choco Creme Biscuits   
4                       Britannia Milk Bikis Biscuits   
34                         Unibic Fruit & Nut Cookies   
19               Sunfeast Marie Light Orange Biscuits   
7                     Britannia Tiger Krunch Biscuits   
12                       Parle Monaco Salted Biscuits   
16                  Sunfeast Marie Light Oat Biscuits   
18             Britannia Milk Bikis Sandwich Biscuits   
23           Sunfeast Marie Light Rich Taste Biscuits   
25     Sunfeast Mom's Magic Cashew & Almonds Biscuits   
0                             Britannia Little Hearts   
1                              Open Secret 1 Gift Box   
3          Sunfeast Bounce Tangy Orange Cream Biscuit   
5                        

127.0.0.1 - - [08/Dec/2020 00:05:21] "POST /_dash-update-component HTTP/1.1" 200 -


                                         Product Name  Category      Brand  \
7                          Parachute Coconut Hair Oil  Hair Oil  Parachute   
10      Parachute Advansed Ayurvedic Hot-Oil Hair Oil  Hair Oil  Parachute   
28                               Dabur Amla  Hair Oil  Hair Oil      Dabur   
35                             Dabur Almond  Hair Oil  Hair Oil      Dabur   
12                    Parachute Pure Coconut Hair Oil  Hair Oil  Parachute   
34     Parachute Advansed Ayurvedic Coconut  Hair Oil  Hair Oil  Parachute   
21  7 Days Unisex Anti Hair Loss Treatment Ginger ...  Hair Oil          7   
3           Parachute Advansed Gold Coconut  Hair Oil  Hair Oil  Parachute   
23                    Biotique Bio Bhringraj Hair Oil  Hair Oil   Biotique   
11                   Navratna Ayurvedic Cool Hair Oil  Hair Oil   Navratna   
15                      Kesh King Ayurvedic  Hair Oil  Hair Oil       Kesh   
27                         Bajaj Brahmi Amla Hair Oil  Hair Oil 